In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import lightgbm as lgb
import geohash2 as geo
import os
from sklearn.metrics import mean_squared_error,explained_variance_score
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',10000)
pd.set_option('display.max_columns',10000)
COLUMNSNAME = ['loadingOrder','carrierName','timestamp','longitude'
               ,'latitude','vesselMMSI','speed','direction','vesselNextport'
               ,'vesselNextportETA','vesselStatus','vesselDatasource','TRANSPORT_TRACE']

In [3]:
def get_data(data, mode='train'):
    
    assert mode=='train' or mode=='test'
    
    if mode=='train':
        data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True)
    elif mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)    
    data['loadingOrder'] = data['loadingOrder'].astype(str)
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    data['longitude'] = data['longitude'].astype(float)
    data['longitude'] = round(data['longitude'],6)
    data['latitude'] = data['latitude'].astype(float)
    data['latitude'] = round(data['latitude'],6)
    data['speed'] = data['speed'].astype(float)
    data['direction'] = data['direction'].astype(float) 
#     data['TRANSPORT_TRACE'] = pd.factorize(data['TRANSPORT_TRACE'])[0]
    return data


In [4]:
def q10(x):
    return x.quantile(0.1)
def q20(x):
    return x.quantile(0.2)
def q30(x):
    return x.quantile(0.3)
def q40(x):
    return x.quantile(0.4)
def q60(x):
    return x.quantile(0.6)
def q70(x):
    return x.quantile(0.7)
def q80(x):
    return x.quantile(0.8)
def q90(x):
    return x.quantile(0.9)

In [ ]:

DATAPATH = 'E:/华为大数据/dataset/train0523.csv'

train_dataset = pd.read_csv(DATAPATH, iterator=True, names=COLUMNSNAME)
train_data = train_dataset.get_chunk(1000).sort_values(by=['loadingOrder','timestamp'],ignore_index=True)

# train_data.loc[train_data['loadingOrder'] == 'ZQ464072113491']
print(train_data['vesselMMSl'].str.contains(r'\w7540547327'))
# str.contains('A')
# train_data = get_data(train_data)

# train_data.head(1000)

# train_data['loadingOrder'].dtypes


按订单全局查找

In [2]:

DATAPATH = 'E:/华为大数据/dataset/train0523.csv'
searchData = pd.read_csv(DATAPATH,chunksize=100000,names=COLUMNSNAME)
search_df = pd.DataFrame(columns=COLUMNSNAME)
for chunk in tqdm(searchData):
    seach = chunk.loc[chunk['loadingOrder'] == 'CE366609955421',:]
#     seach = chunk.loc[chunk['vesselMMSl'].str.contains(r'\w7540547327'),:]
    search_df = search_df.append(seach)
search_df = search_df.sort_values(by=['loadingOrder','timestamp'], ignore_index=True)
search_df

1480it [03:43,  6.63it/s]


,loadingOrder,carrierName,timestamp,longitude,latitude,vesselMMSI,speed,direction,vesselNextport,vesselNextportETA,vesselStatus,vesselDatasource,TRANSPORT_TRACE
0,CE366609955421,OIEQNT,2020-04-29T12:21:43.000Z,114.268942,22.565530,M8022557824,11,11990,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
1,CE366609955421,OIEQNT,2020-04-29T12:26:22.000Z,114.279048,22.563327,M8022557824,15,9020,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
2,CE366609955421,OIEQNT,2020-04-29T12:29:14.000Z,114.286500,22.563395,M8022557824,18,9110,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
3,CE366609955421,OIEQNT,2020-04-29T12:31:23.000Z,114.293205,22.563285,M8022557824,19,9130,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
4,CE366609955421,OIEQNT,2020-04-29T12:33:44.000Z,114.301270,22.563207,M8022557824,20,9080,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
5,CE366609955421,OIEQNT,2020-04-29T12:35:44.000Z,114.307250,22.563123,M8022557824,19,9130,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
6,CE366609955421,OIEQNT,2020-04-29T12:39:54.000Z,114.319662,22.562940,M8022557824,17,9010,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
7,CE366609955421,OIEQNT,2020-04-29T12:44:04.000Z,114.330845,22.562828,M8022557824,17,9060,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
8,CE366609955421,OIEQNT,2020-04-29T12:47:23.000Z,114.341177,22.560615,M8022557824,19,11160,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX
9,CE366609955421,OIEQNT,2020-04-29T12:51:44.000Z,114.355288,22.555495,M8022557824,23,11170,CN YTN>PA PCN,2020-05-19T18:00:00.000Z,under way using engine,Coastal AIS,CNYTN-PAONX


In [23]:
search_df.shape

(4755, 13)

,mmax,mmin
loadingOrder,,
AA236935066990,2019-09-13T12:15:51.000Z,2019-08-12T11:15:50.000Z


In [ ]:

savefilePath = 'E:/华为大数据/'
# alphabat = ['A','B','C','D','E','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
alphabat = ['A']
def orderTocsv():  
    train_df=pd.DataFrame(columns=COLUMNSNAME)
    for alpha in alphabat:
        chunk_list=[]
        train_flux = pd.read_csv(DATAPATH,chunksize=10000,names=COLUMNSNAME)
        for chunk in tqdm(train_flux):
    #         data = chunk.loc[chunk['loadingOrder'] == order,:]
            data = chunk.loc[chunk['loadingOrder'].str.contains(r'^[{}]'.format(alpha), regex=True)]
            chunk_list.append(data)

        train_df = pd.concat(chunk_list)
        train_df = train_df.sort_values(by=['loadingOrder','timestamp'],ignore_index=True)
        orderfile = 'train_'+alpha+'.csv'
        train_df.to_csv(savefilePath+orderfile, columns=COLUMNSNAME)
    #         train_df.drop(train_df.index,inplace=True)
        
orderTocsv()
    

In [ ]:
testTracelist = ['CNYTN-MXZLO','CNSHK-MYTPP','CNSHK-SGSIN','CNSHK-CLVAP','CNYTN-ARENA',
                 'CNYTN-MATNG','CNSHK-GRPIR','CNSHK-PKQCT','COBUN-HKHKG','CNYTN-PAONX',
                 'CNSHK-SIKOP','CNYTN-CAVAN','CNSHK-ESALG','CNYTN-MTMLA','CNSHK-ZADUR',
                 'CNSHK-LBBEY','CNSHA-SGSIN','CNYTN-RTM','CNHKG-MXZLO','HKHKG-FRFOS',
                 'CNYTN-NZAKL','CNSHA-PAMIT']
port = []
splitTrace = pd.read_csv('E:/traindata/train_50wan.csv')
traceNan = splitTrace['TRANSPORT_TRACE'].isna()
tracedroplist = traceNan.loc[traceNan.values].index.tolist()

splitTrace = splitTrace.drop(tracedroplist)
for trace in testTracelist:
    port.append(trace.split('-'))
    
# splitTrace = splitTrace.append(splitTrace.loc[splitTrace['TRANSPORT_TRACE'].str.contains(regex)])
# splitTrace.head(100)    


去除方向-1，trace>2和Nan的数据订单，并前向填充订单中ETA等的空值

In [3]:
filelist = ['train_{}.csv'.format(i) for i in ['B','C','D','E','F','G','H','I','J','K','L','M','N','O','P',
                                               'Q','R','S','T','U','V','W','X','Y', 'Z']]

testTracelist = ['CNYTN-MXZLO','CNSHK-MYTPP','CNSHK-SGSIN','CNSHK-CLVAP','CNYTN-ARENA',
                 'CNYTN-MATNG','CNSHK-GRPIR','CNSHK-PKQCT','COBUN-HKHKG','CNYTN-PAONX',
                 'CNSHK-SIKOP','CNYTN-CAVAN','CNSHK-ESALG','CNYTN-MTMLA','CNSHK-ZADUR',
                 'CNSHK-LBBEY','CNSHA-SGSIN','CNYTN-RTM','CNHKG-MXZLO','HKHKG-FRFOS',
                 'CNYTN-NZAKL','CNSHA-PAMIT']

# filelist = ['train_A.csv']
                                          
for FILENAME in tqdm(filelist):    

    srcDATAPATH = 'E:/traindata/source/' + FILENAME
    dstDATAPATH = 'E:/traindata/repaired/' + FILENAME
            
# srcDATAPATH = 'E:/traindata/train_50wan.csv'
# trainData = pd.read_csv(srcDATAPATH)

    trainData = pd.read_csv(srcDATAPATH,index_col=0)
#   trainData = get_data(trainData,mode='train')

    # trainData.isnull().sum()

    #     etaNan = trainData.groupby('loadingOrder')['vesselNextportETA'].max().isna()
    #     etadroplist = etaNan.loc[etaNan.values].index.tolist()
    #     for i in tqdm(etadroplist):
    #         dropindex = trainData.loc[trainData['loadingOrder'] == i,:].index.tolist()
    #     print(dropindex)
    #         trainData = trainData.drop(dropindex)
    
    traceNan = trainData['TRANSPORT_TRACE'].isna()
    tracedroplist = traceNan.loc[traceNan.values].index.tolist()
    trainData = trainData.drop(tracedroplist)

    traceMoreThan3 = trainData.loc[trainData['TRANSPORT_TRACE'].str.contains(r'\w+-\w+-.',regex=True)].index.tolist()
    trainData = trainData.drop(traceMoreThan3)

    #     trainData = trainData[trainData['TRANSPORT_TRACE'].isin(testTracelist)]

    speed0 = trainData.loc[trainData['speed'] >= 90].index.tolist()
    trainData = trainData.drop(speed0)
    
    directionNeg1 = trainData.loc[trainData['direction'] == -1].index.tolist()
    trainData = trainData.drop(directionNeg1)
    
    trainData = trainData.reset_index(drop=True)
    
#     trainData['diff_seconds'] = trainData.groupby('loadingOrder')['timestamp'].diff(1).dt.total_seconds()
#     delminsec = trainData.loc[trainData['diff_seconds'] <= 20].index.tolist()
#     trainData = testData.drop(delminsec)   
       
    trainData.vesselNextport = trainData.vesselNextport.fillna(method='bfill')
    trainData.vesselNextportETA = trainData.vesselNextportETA.fillna(method='bfill')
    trainData.vesselStatus = trainData.vesselStatus.fillna(method='bfill')

    # trainData.isna().sum()
    trainData.to_csv(dstDATAPATH,index=False)
#     trainData.tail(400)


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [12:13<00:00, 29.32s/it]


In [ ]:
filelist = ['train_{}.csv'.format(i) for i in ['B','C','D','E','F','G','H','I','J','K','L',
                                           'M','N','O','P','Q','R','S','T','U','V','W','X',
                                           'Y','Z']]
# filelist = ['train_A.csv']

for filename in tqdm(filelist):
    dataPath = 'E:/traindata/repaired/' + filename
    train_data = pd.read_csv(dataPath)
    train_data = get_data(train_data,mode='train')
    train_data['diff_seconds'] = train_data.groupby('loadingOrder')['timestamp'].diff(1).dt.total_seconds()
    delminsec = train_data.loc[train_data['diff_seconds'] <= 30].index.tolist()
    train_data = train_data.drop(delminsec) 
    train_data = train_data.drop(columns = ['diff_seconds'])
    train_data = train_data.reset_index(drop=True)
#     train_data.to_csv(dataPath, index=False)

In [ ]:
filelist = ['train_{}.csv'.format(i) for i in ['D','E','F','G','H','I','J','K','L',
                                           'M','N','O','P','Q','R','S','T','U','V','W','X',
                                           'Y','Z']]

for filename in filelist:
    dataPath = 'E:/traindata/repaired/'+filename
    data = pd.read_csv(dataPath)
    dataorder = data.groupby('loadingOrder').size()

    dropleslist =dataorder.loc[dataorder.values < 100].index.tolist() 
    for order in dropleslist:
        dropindex = data.loc[data['loadingOrder'] == order,:].index.tolist()
        data = data.drop(dropindex)
    data = data.reset_index(drop=True)
    data.to_csv(dataPath,index=False)

geohash 特征

In [5]:
def get_geo_feature(df,num):
    tmp = df.groupby('loadingOrder')['geo'].agg(list).reset_index()
    tmp['geo'] = tmp['geo'].apply(lambda x: ' '.join(x))
    
    tfidf_enc_tmp = TfidfVectorizer()
    tfidf_vec_tmp = tfidf_enc_tmp.fit_transform(tmp['geo'])
    svd_tag_tmp = TruncatedSVD(n_components=num, n_iter=20, random_state=1024)
    tag_svd_tmp =svd_tag_tmp.fit_transform(tfidf_vec_tmp)
    tag_svd_tmp = pd.DataFrame(tag_svd_tmp)
    tag_svd_tmp.columns = ['geo_tfidf_{}'.format(i) for i in range(num)]
    
    countvec = CountVectorizer()
    count_vec_tmp = countvec.fit_transform(tmp['geo'])
    svd_tmp = TruncatedSVD(n_components=num, n_iter=20, random_state=1024)
    svd_tmp = svd_tmp.fit_transform(count_vec_tmp)
    svd_tmp = pd.DataFrame(svd_tmp)
    svd_tmp.columns = ['geo_countvec_{}'.format(i) for i in range(num)]
    return pd.concat([tmp[['loadingOrder']], tag_svd_tmp, svd_tmp], axis=1)

# geodata = pd.read_csv('E:/traindata/repaired/train_K.csv')
# geodata = get_geo_feature(geodata,20)
# geodata.head(10)    

梯度特征

In [6]:
def get_grad_feature(df,num):
    grad_df = df.groupby('loadingOrder')['latitude'].apply(lambda x:np.gradient(x)).reset_index()
    grad_df['longitude'] = df.groupby('loadingOrder')['longitude'].apply(lambda x:np.gradient(x)).reset_index()['longitude']
    grad_df['latitude'] = grad_df['latitude'].apply(lambda x: np.round(x, 4))
    grad_df['longitude'] = grad_df['longitude'].apply(lambda x: np.round(x, 4))
    grad_df['grad'] = grad_df.apply(lambda x: ' '.join(['{}_{}'.format(z[0], z[1]) for z in zip(x['latitude'], x['longitude'])]),axis=1)
    
    tfidf_enc_tmp = TfidfVectorizer()
    tfidf_vec_tmp = tfidf_enc_tmp.fit_transform(grad_df['grad'])
    svd_tag_tmp = TruncatedSVD(n_components=num, n_iter=20, random_state=1024)
    tag_svd_tmp = svd_tag_tmp.fit_transform(tfidf_vec_tmp)
    tag_svd_tmp = pd.DataFrame(tag_svd_tmp)
    tag_svd_tmp.columns = ['grad_tfidf_{}'.format(i) for i in range(num)]
    return pd.concat([grad_df[['loadingOrder']], tag_svd_tmp], axis=1)

# graddata = pd.read_csv('E:/traindata/repaired/train_K.csv')
# graddata = get_grad_feature(graddata,20)
# graddata.head(10)       

sample 特征

In [7]:
def get_sample_feature(df, num):
    tmp = df.groupby('loadingOrder')['geo'].apply(lambda x: x.sample(frac=0.1, random_state=1)).reset_index()
    del tmp['level_1']
    tmp.columns = ['loadingOrder', 'sample']
    tmp = tmp.groupby('loadingOrder')['sample'].agg(list).reset_index()
    tmp['sample'] = tmp['sample'].apply(lambda x: ' '.join(x))
    
    tfidf_enc_tmp = TfidfVectorizer()
    tfidf_vec_tmp = tfidf_enc_tmp.fit_transform(tmp['sample'])
    svd_tag_tmp = TruncatedSVD(n_components=num, n_iter=20, random_state=1024)
    tag_svd_tmp = svd_tag_tmp.fit_transform(tfidf_vec_tmp)
    tag_svd_tmp = pd.DataFrame(tag_svd_tmp)
    tag_svd_tmp.columns = ['sample_tfidf_{}'.format(i)
                           for i in range(num)]
    
    return pd.concat([tmp[['loadingOrder']], tag_svd_tmp], axis=1)
        
# sampledata = pd.read_csv('E:/traindata/repaired/train_K.csv')
# sampledata = get_sample_feature(sampledata,20)
# sampledata.head(10)
    

w2v特征

In [2]:
def hashfxn(astring):
    return ord(astring[0])

def w2v_feature(df, length):
    data_frame = df.groupby('loadingOrder')['geo'].agg(list).reset_index()
    model = Word2Vec(data_frame['geo'].values, size=length, window=5, min_count=1, workers=1, iter=10, seed=1, hashfxn=hashfxn)
    data_frame['geo'] = data_frame['geo'].apply(lambda x: pd.DataFrame([model[c] for c in x]))
    for m in range(length):
        data_frame['w2v_{}_mean'.format(m)] = data_frame['geo'].apply(lambda x: x[m].mean())
    del data_frame['geo']
    return data_frame

w2vdata = pd.read_csv('E:/traindata/repaired/train_K.csv')
w2vdata = w2v_feature(w2vdata,15)
w2vdata.head(10)


,loadingOrder,w2v_0_mean,w2v_1_mean,w2v_2_mean,w2v_3_mean,w2v_4_mean,w2v_5_mean,w2v_6_mean,w2v_7_mean,w2v_8_mean,w2v_9_mean,w2v_10_mean,w2v_11_mean,w2v_12_mean,w2v_13_mean,w2v_14_mean
0,KA201061677965,0.531203,0.795341,-1.282222,-0.134039,0.506907,-1.608460,-1.180437,-0.899561,1.561953,-1.446333,0.285931,-0.333491,0.494634,-0.929576,0.191347
1,KA414697694760,0.930997,0.394813,-2.258932,0.665913,1.112202,-1.854313,-1.503237,-0.497763,1.678776,-2.482710,0.338439,-0.048037,1.225356,-1.452074,0.436067
2,KA598870513950,1.007447,0.688684,-2.171681,0.956929,0.725221,-2.496249,-2.242985,-1.276935,1.971210,-2.689570,0.486342,0.094102,1.728470,-1.904430,-0.021628
3,KA633912017824,1.471709,1.029400,-1.696654,-2.754118,-0.402644,-2.923530,-3.465180,-3.022854,4.113226,-2.228606,0.778483,0.434441,1.258540,-2.823944,-2.152489
4,KA666439456356,1.001174,0.603803,-2.293690,0.917571,1.514090,-2.614501,-1.338580,-0.752032,1.905579,-2.586739,0.175372,-0.321154,1.445923,-1.929814,0.351992
5,KA733440782308,0.834833,1.604048,-4.902457,1.364584,2.756300,-5.904866,-2.758568,-2.099140,3.637970,-5.140977,1.543843,-0.043071,2.735212,-4.811028,0.911264
6,KA785275274461,1.556402,1.170340,-1.304698,-2.147821,-0.072342,-3.976833,-2.929629,-3.325923,4.518992,-1.839488,0.157498,-0.002051,1.161699,-3.298608,-2.522712
7,KB404388532408,0.962263,0.644551,-1.130752,0.641879,0.638474,-1.379027,-1.012408,-0.925091,1.442754,-1.621204,0.762425,-0.731438,0.873502,-1.082022,0.180924
8,KB494965502600,1.538785,1.243205,-2.715907,0.341365,1.833841,-3.332581,-1.761317,-1.539222,2.995568,-3.477812,0.378710,-1.020299,1.750420,-2.786535,0.022441
9,KB591162326411,1.837305,1.335540,-3.533401,0.989091,2.626356,-4.244023,-2.193951,-1.360788,3.333705,-4.343426,0.497874,-1.227104,2.453552,-3.152680,0.512023


特征提取

In [9]:
def get_feature(df, mode='train'):
    
    assert mode == 'train' or mode == 'test'
    
    df['diff_lat'] = df.groupby('loadingOrder')['latitude'].diff(1)
    df['diff_lon'] = df.groupby('loadingOrder')['longitude'].diff(1)
    df['diff_speed'] = df.groupby('loadingOrder')['speed'].diff(1)
    df['diff_minutes'] = df.groupby('loadingOrder')['timestamp'].diff(1).dt.total_seconds() // 60
#     df['anchor'] = df.apply(lambda x: 1 if x['diff_lat'] <= 0.03 and x['diff_lon'] <= 0.03
#                             and x['diff_speed'] <= 0.3 and x['diff_minutes'] <= 10 else 0, axis=1)
    df['anchor'] = ((df['diff_lat']<=0.03)&(df['diff_lon']<=0.03)&(df['diff_speed']<=0.3)&(df['diff_minutes']<=10)).astype('int')
    
    if mode=='train':
        group_df = df.groupby('loadingOrder')['timestamp'].agg(mmax='max', count='count', mmin='min')
        group_df['label'] = (group_df['mmax'] - group_df['mmin']).dt.total_seconds() // 3600
#         group_df['trace'] = df.groupby('loadingOrder')['TRANSPORT_TRACE'].max()
    elif mode=='test':        
    # test数据处理
        group_df = df.groupby('loadingOrder')['timestamp'].agg(count='count').reset_index()
#         group_df['trace'] = df.groupby('loadingOrder')['TRANSPORT_TRACE'].max()
#         group_df = group_df.reset_index()
    
    gt_zero = df[(df['diff_lat'] != 0)&(df['diff_lon'] != 0)]
    speed_gt_zero = df[df['diff_speed'] != 0]
    
    
    anchor_df = df.groupby('loadingOrder')['anchor'].agg('sum').reset_index()
    anchor_df.columns = ['loadingOrder','anchor_cnt']
    group_df = group_df.merge(anchor_df, on='loadingOrder', how='left')
    group_df['anchor_ratio'] = group_df['anchor_cnt'] / group_df['count']
    
    
    agg_function = ['min', 'max', 'mean', 'median','nunique', q10, q20, q30, q40, q60, q70, q80, q90]
    agg_ways = ['min', 'max', 'mean', 'median', 'nunique', 'q_10', 'q_20', 'q_30', 'q_40', 'q_60', 'q_70', 'q_80', 'q_90']
    
    agg_col = ['latitude', 'longitude', 'speed', 'direction']
    group = df.groupby('loadingOrder')[agg_col].agg(agg_function).reset_index()
    group.columns = ['loadingOrder'] + ['{}_{}'.format(i, j) for i in agg_col for j in agg_ways]
    
    gt_zero_group = gt_zero.groupby('loadingOrder')[agg_col].agg(agg_function).reset_index()
    gt_zero_group.columns = ['loadingOrder'] + ['pos_gt_zero_{}_{}'.format(i, j) for i in agg_col for j in agg_ways]
    
    speed_gt_zero_group = speed_gt_zero.groupby('loadingOrder')[agg_col].agg(agg_function).reset_index()
    speed_gt_zero_group.columns = ['loadingOrder'] + ['speed_gt_zero_{}_{}'.format(i, j) for i in agg_col for j in agg_ways]
      
    group_df = group_df.merge(group, on='loadingOrder', how='left')
    group_df = group_df.merge(gt_zero_group, on='loadingOrder', how='left')
    group_df = group_df.merge(speed_gt_zero_group, on='loadingOrder', how='left')
    
    tfidf_df = get_geo_feature(df,15)
    group_df = group_df.merge(tfidf_df, on='loadingOrder', how='left')
    
    grad_df = get_grad_feature(df,15)
    group_df = group_df.merge(grad_df, on='loadingOrder', how='left')
    
    sample_df = get_sample_feature(df,15)
    group_df = group_df.merge(sample_df, on='loadingOrder', how='left')
    
    w2v_df = w2v_feature(df, 15)
    group_df = group_df.merge(w2v_df, on='loadingOrder', how='left')
    
    return group_df

距离

In [22]:
def distance(latA,latB,lonA,lonB):
    EARTH_RADIUS = 6378.137 # 千米
    def rad(d):
        return d * np.pi/ 180.0
    s=0
    radLatA = rad(latA)
    radLatB = rad(latB)
    a = radLatA - radLatB
    b = rad(lonA) - rad(lonB)
    s = 2 * np.arcsin(np.sqrt(np.power(np.sin(a / 2),2)+ np.cos(radLatA) * np.cos(radLatB)*np.power(np.sin(b / 2),2)))
    s = s * EARTH_RADIUS
    #  保留两位小数
    s = np.round(s * 100)/100
    s = s * 1000 # 转换成m
    return s
distance(140.144815,150.760973,41.254007,44.942478)

1228900.0

In [ ]:
srcDATAPATH = 'E:/traindata/source/train_A.csv'
testtest = pd.read_csv(srcDATAPATH)
testtest = get_data(testtest)
speed0 = testtest.loc[testtest['speed'] > 90].index.tolist()
testtest['diff_seconds'] = testtest.groupby('loadingOrder')['timestamp'].diff(1).dt.total_seconds()

diffmin = testtest.loc[testtest['diff_seconds'] < 20].index.tolist()
testtest = testtest.drop(diffmin)
testtest
# testtest.head(10000)
# testtest = get_feature(testtest)
# testtest.head(100)
# speed0

创建训练集

In [ ]:
filelist = ['train_{}.csv'.format(i) for i in ['B','C','D','E','F','G','H','I','J','K','L',
                                               'M','N','O','P','Q','R','S','T','U','V',
                                               'W','X','Y','Z']]
# filelist = ['train_A.csv']
for item in tqdm(filelist):

    DATAPATH = 'E:/traindata/repaired/' + item
    TDPATH = 'E:/traindata/train1.csv'

    trainData = pd.read_csv(DATAPATH)
    trainData = get_data(trainData, mode='train')

    featureData = get_feature(trainData)
    featureData.to_csv(TDPATH, mode='a',index=False,header=False)


创建测试集

In [ ]:
TESTDATA = 'E:/traindata/test/testdata.csv'
TESTPATH = 'E:/traindata/test1.csv'

testData = pd.read_csv(TESTDATA)
testData = get_data(testData, mode='test')
# tracelist = testData.TRANSPORT_TRACE.unique().tolist()
# tracelist

testData = get_feature(testData,mode='test')
# testData.tail(100)
testData.to_csv(TESTPATH,index=False)

In [19]:
data = pd.read_csv('E:/traindata/train.csv')
# data['label'] = data['label'] // 3600
data

,loadingOrder,timestamp,longitude,latitude,vesselMMSI,speed,direction,vesselNextportETA,TRANSPORT_TRACE,diff_lat,diff_lon,diff_speed,diff_minutes,diff_direction,trace
0,AA191175561416,2019-01-28 16:12:59+00:00,114.260392,22.571047,Y7540547327,0.0,12670.0,2019-01-28 05:30:00+00:00,CNYTN-MXZLO,NaN,NaN,NaN,NaN,NaN,5
1,AA191175561416,2019-01-28 16:22:38+00:00,114.260438,22.571125,Y7540547327,0.0,14790.0,2019-01-28 05:30:00+00:00,CNYTN-MXZLO,0.000078,0.000046,0.0,9.0,21.2,5
2,AA191175561416,2019-01-28 16:30:55+00:00,114.260693,22.571567,Y7540547327,0.0,21510.0,2019-01-28 05:30:00+00:00,CNYTN-MXZLO,0.000442,0.000255,0.0,8.0,67.2,5
3,AA191175561416,2019-01-28 16:37:35+00:00,114.260392,22.571463,Y7540547327,0.0,19900.0,2019-01-28 05:30:00+00:00,CNYTN-MXZLO,-0.000104,-0.000301,0.0,6.0,16.1,5
4,AA191175561416,2019-01-28 16:45:56+00:00,114.260647,22.571510,Y7540547327,0.0,21360.0,2019-01-28 05:30:00+00:00,CNYTN-MXZLO,0.000047,0.000255,0.0,8.0,14.6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571225,ZY989876486292,2019-04-26 19:05:20+00:00,-104.322813,19.066443,D7828192902,11.0,11170.0,2019-04-26 06:00:00+00:00,CNYTN-MXZLO,-0.001380,0.006139,0.0,3.0,24.9,1
571226,ZY989876486292,2019-04-26 19:11:00+00:00,-104.311943,19.061608,D7828192902,15.0,12500.0,2019-04-26 06:00:00+00:00,CNYTN-MXZLO,-0.004835,0.010870,4.0,5.0,13.3,1
571227,ZY989876486292,2019-04-26 19:12:49+00:00,-104.308012,19.060088,D7828192902,14.0,9650.0,2019-04-26 06:00:00+00:00,CNYTN-MXZLO,-0.001520,0.003931,-1.0,1.0,28.5,1
571228,ZY989876486292,2019-04-26 19:36:40+00:00,-104.296138,19.058382,D7828192902,1.0,20150.0,2019-04-26 06:00:00+00:00,CNYTN-MXZLO,-0.001706,0.011874,-13.0,23.0,105.0,1


In [ ]:
traindatapath = 'E:/traindata/train1.csv'
testdatapath = 'E:/traindata/test1.csv'

traindata = pd.read_csv(traindatapath)
testdata = pd.read_csv(testdatapath)

traindata = traindata.drop(columns='trace')
testdata = testdata.drop(columns='trace')

traindata.to_csv(traindatapath,index=False)
testdata.to_csv(testdatapath,index=False)


模型训练

In [16]:
def mse_score_eval(preds, valid):
    labels = valid.get_label()
    scores = mean_squared_error(y_true=labels,y_pred=preds)
    return 'mse_score',scores,True
def build_model(train, test, pred, label, seed=2020, is_shuffle=True):
    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 5
    # Kfold
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    kf_way = fold.split(train[pred])
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 63,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.7,
        'bagging_freq': 6,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 8,
        'verbose': -1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        train_x, train_y = train[pred].iloc[train_idx], train[label].iloc[train_idx]
        valid_x, valid_y = train[pred].iloc[valid_idx], train[label].iloc[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=n_train,
            num_boost_round=3000,
            valid_sets=[n_valid],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=mse_score_eval
        )
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test[pred], num_iteration=clf.best_iteration)/fold.n_splits
    
    test['label'] = test_pred
    
    return test[['loadingOrder', 'label']]

train = pd.read_csv('E:/traindata/train1.csv')
test = pd.read_csv('E:/traindata/test1.csv')
features = [c for c in train.columns if c not in ['loadingOrder', 'label', 'mmin', 'mmax', 'count']]

# print(features)
result = build_model(train, test, features,'label',is_shuffle=True)
result


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4276.77	valid_0's mse_score: 4276.77
Early stopping, best iteration is:
[1]	valid_0's l2: 74291.5	valid_0's mse_score: 74291.5
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4189.19	valid_0's mse_score: 4189.19
Early stopping, best iteration is:
[1]	valid_0's l2: 68913.7	valid_0's mse_score: 68913.7
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4923.18	valid_0's mse_score: 4923.18
Early stopping, best iteration is:
[1]	valid_0's l2: 72719.5	valid_0's mse_score: 72719.5
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 3751.08	valid_0's mse_score: 3751.08
Early stopping, best iteration is:
[1]	valid_0's l2: 69203	valid_0's mse_score: 69203
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 4381.55	valid_0's mse_score: 4381.55
Early stopping, best iteration is:
[1]	valid_0's 

,loadingOrder,label
0,CF946210847851,412.573954
1,CI265639541482,413.217496
2,CJ649212973636,412.573954
3,CM247448368157,412.573954
4,CX921020302440,412.573954
5,CY610195685233,412.573954
6,CY693700885314,412.573954
7,DN799599926798,412.573954
8,DQ469582150123,412.573954
9,DS133295450477,412.573954


In [17]:
d = {'CNHKG-MXZLO':2210541,'CNSHA-PAMIT':886676,'CNSHK-CLVAP':2178500,'CNSHK-LBBEY':488011,'CNSHA-SGSIN':1269200,
'CNSHK-PKQCT':1253011,'CNSHK-SGSIN':375811,'CNSHK-SIKOP':1861884,'CNSHK-ZADUR':1851936,'CNSHK-ESALG':1269200,'HKHKG-FRFOS':1269200,
'CNYTN-ARENA':2421561,'CNYTN-CAVAN':1178228,'CNYTN-MATNG':1818090,'CNYTN-MTMLA':3084132,'CNSHK-GRPIR':1269200,'CNSHK-MYTPP':1269200,
'CNYTN-MXZLO':1813206,'CNYTN-NZAKL':1128508,'CNYTN-PAONX':1717306,'CNYTN-RTM':2025894,'COBUN-HKHKG':3870274}

TESTDATA = 'E:/traindata/test/A_testData0531.csv'

test_data = pd.read_csv(TESTDATA)
test_data = get_data(test_data,mode='test')
test_data = test_data.merge(result, on='loadingOrder', how='left')

# group_data = test_data.groupby('TRANSPORT_TRACE')['label']
# for index,value in group_data:
#     value = value + d[index]//4
# #     v.append(value)
#     test_data.loc[test_data['TRANSPORT_TRACE'] == index,'label'] = value

test_data['ETA'] = (test_data['onboardDate'] + test_data['label'].apply(lambda x:pd.Timedelta(hours=x))).apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = test_data['temp_timestamp']
# 整理columns顺序
result = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]
result.to_csv('result.csv',index=False)


In [3]:
#休闲一刻猜分数
gueesData = pd.read_csv('result.csv')
gueesData['ETA'] = (pd.to_datetime(gueesData['ETA']) - pd.Timedelta(hours=70)).apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
# gueesData.head(10)
gueesData['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
# gueesData.head(10)
gueesData.to_csv('result1.csv',index=False)

In [ ]:
TESTDATA = 'E:/traindata/test/A_testData0531.csv'

d = {'CNHKG-MXZLO':2210541,'CNSHA-PAMIT':886676,'CNSHK-CLVAP':2178500,'CNSHK-LBBEY':488011,'CNSHA-SGSIN':1269200,
'CNSHK-PKQCT':1253011,'CNSHK-SGSIN':375811,'CNSHK-SIKOP':1861884,'CNSHK-ZADUR':1851936,'CNSHK-ESALG':1269200,'HKHKG-FRFOS':1269200,
'CNYTN-ARENA':2421561,'CNYTN-CAVAN':1178228,'CNYTN-MATNG':1818090,'CNYTN-MTMLA':3084132,'CNSHK-GRPIR':1269200,'CNSHK-MYTPP':1269200,
'CNYTN-MXZLO':1813206,'CNYTN-NZAKL':1128508,'CNYTN-PAONX':1717306,'CNYTN-RTM':2025894,'COBUN-HKHKG':3870274}

test_data = pd.read_csv(TESTDATA)
test_data = get_data(test_data,mode='test')
test_data = test_data.merge(result, on='loadingOrder', how='left')
group_data = test_data.groupby('TRANSPORT_TRACE')['label']



for index,value in group_data:
    value = value + d[index]//4
#     v.append(value)
    test_data.loc[test_data['TRANSPORT_TRACE'] == index,'label'] = value

test_data

In [ ]:
trainPath = 'E:/traindata/train.csv'
filename = 'E:/traindata/port.csv'
data = pd.read_csv(filename)
data = data.groupby('TRANSPORT_TRACE')['loadingOrder'].unique().index
for d in data:
    train = pd.read_csv(trainPath)
    train = train.loc[train['loadingOrder'].isin(d)]
    print(int(train['label'].mean()))
data 


geohash 编码

In [ ]:
__all__ = ['encode','decode','bbox','neighbors']
_base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
#10进制和32进制转换，32进制去掉了ailo
_decode_map = {}
_encode_map = {}
for i in range(len(_base32)):
    _decode_map[_base32[i]] = i
    _encode_map[i]=_base32[i]
del i

def encode(lat,lon,precision=12):
    lat_range, lon_range = [-90.0, 90.0], [-180.0, 180.0]
    geohash=[]
    code=[]
    while len(geohash)<precision:
#         print(code,lat_range,lon_range,geohash)
        lat_mid=sum(lat_range)/2
        lon_mid=sum(lon_range)/2
        #经度
        if lon<lon_mid:
            code.append(0)
            lon_range[1]=lon_mid
        else:
            code.append(1)
            lon_range[0]=lon_mid
        #纬度
        if lat<lat_mid:
            code.append(0)
            lat_range[1]=lat_mid
        else:
            code.append(1)
            lat_range[0]=lat_mid
        ##encode
        if len(code)>=5:
            geohash.append(_encode_map[int(''.join(map(str,code[:5])),2)])
            code=code[5:]
    return ''.join(geohash)

encode(39.928167,116.389550,6)


In [47]:
# filelist = ['train_{}.csv'.format(i) for i in ['B','C','D','E','F','G','H','I','J','K','L',
#                                            'M','N','O','P','Q','R','S','T','U','V','W','X',
#                                            'Y','Z']]
# filelist = ['train_A.csv']


# for FILENAME in filelist:
    
srcDATAPATH = 'E:/traindata/B/halfdata.csv'
#     srcDATAPATH = 'E:/traindata/repaired/' + FILENAME
CHUNKSIZE = 10000
train_data = pd.read_csv(srcDATAPATH,chunksize=CHUNKSIZE)
geolist = []
for chunk in tqdm(train_data):
#     data = get_data(chunk,mode='train')
    lat = chunk.latitude.tolist()
    lon = chunk.longitude.tolist()
    arr = np.array([lat,lon])
    for i in range(len(chunk)):
        geolist.append((geo.encode(arr[0,i],arr[1,i],4)))
#     geoseries = pd.Series(geolist)
geoData = pd.read_csv(srcDATAPATH)
# geoData.insert(5, 'port1geo', geolist)
geoData['geo'] = geolist
geoData = geoData.reset_index(drop=1)
geoData.head(10)

309it [00:32,  9.38it/s]


,loadingOrder,carrierName,timestamp,longitude,latitude,vesselMMSI,speed,direction,vesselNextport,vesselNextportETA,vesselStatus,vesselDatasource,TRANSPORT_TRACE,geo
0,AA191175561416,OIEQNT,2019-01-28T16:12:59.000Z,114.260392,22.571047,Y7540547327,0,12670,HK HKG >CN YTN,2019-01-28T05:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
1,AA191175561416,OIEQNT,2019-01-28T16:22:38.000Z,114.260438,22.571125,Y7540547327,0,14790,HK HKG >CN YTN,2019-01-28T05:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
2,AA191175561416,OIEQNT,2019-01-28T16:30:55.000Z,114.260693,22.571567,Y7540547327,0,21510,HK HKG >CN YTN,2019-01-28T05:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
3,AA191175561416,OIEQNT,2019-01-28T16:37:35.000Z,114.260392,22.571463,Y7540547327,0,19900,HK HKG >CN YTN,2019-01-28T05:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
4,AA191175561416,OIEQNT,2019-01-28T16:45:56.000Z,114.260647,22.571510,Y7540547327,0,21360,HK HKG >CN YTN,2019-01-28T05:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
5,AA191175561416,OIEQNT,2019-01-28T16:52:34.000Z,114.260407,22.571173,Y7540547327,0,13950,CN YTN >TW KHH,2019-01-29T17:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
6,AA191175561416,OIEQNT,2019-01-28T17:04:36.000Z,114.260827,22.571203,Y7540547327,0,21010,CN YTN >TW KHH,2019-01-29T17:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
7,AA191175561416,OIEQNT,2019-01-28T17:21:57.000Z,114.260638,22.571392,Y7540547327,0,13090,CN YTN >TW KHH,2019-01-29T17:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
8,AA191175561416,OIEQNT,2019-01-28T17:36:58.000Z,114.260573,22.571360,Y7540547327,0,7630,CN YTN >TW KHH,2019-01-29T17:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12
9,AA191175561416,OIEQNT,2019-01-28T17:49:34.000Z,114.260307,22.570985,Y7540547327,0,5310,CN YTN >TW KHH,2019-01-29T17:30:00.000Z,moored,Coastal AIS,CNYTN-MXZLO,ws12


In [54]:
# geodf['timestamp'] = pd.to_datetime(geodf['timestamp'], infer_datetime_format=True)
# group_df = geodf.groupby('loadingOrder')['timestamp'].agg(mmax='max', mmin='min')
# group_df['label'] = (group_df['mmax'] - group_df['mmin']).dt.total_seconds() / 3600
# group_df
geodf = geoData.drop_duplicates(['loadingOrder','geo'],keep='last')
geodf = geodf.reset_index(drop=1)
geodf.TRANSPORT_TRACE = geodf.TRANSPORT_TRACE.astype(str)

portlist = geodf['TRANSPORT_TRACE'].unique().tolist()
droptracelist = []
for ls in portlist:
    tr = ls.split('-')
    if len(tr) == 1 :
        droptracelist.append(ls)
geodf = geodf[~geodf['TRANSPORT_TRACE'].isin(droptracelist)].reset_index(drop=1)

geodf['port0name'] = geodf['TRANSPORT_TRACE'].apply(lambda x: x.split('-')[0])
geodf['port1name'] = geodf['TRANSPORT_TRACE'].apply(lambda x: x.split('-')[-1])
geodf.to_csv('E:/traindata/B/halfdata_geo4.csv',index=False)
geodf.head(10)

,loadingOrder,carrierName,timestamp,longitude,latitude,vesselMMSI,speed,direction,vesselNextport,vesselNextportETA,vesselStatus,vesselDatasource,TRANSPORT_TRACE,geo,port0name,port1name
0,AA191175561416,OIEQNT,2019-01-29T15:51:14.000Z,114.424042,22.503205,Y7540547327,25,13730,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,ws12,CNYTN,MXZLO
1,AA191175561416,OIEQNT,2019-01-29T16:46:55.000Z,114.567383,22.331025,Y7540547327,35,12500,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wecr,CNYTN,MXZLO
2,AA191175561416,OIEQNT,2019-01-29T17:42:20.000Z,114.835465,22.158043,Y7540547327,34,12740,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wecw,CNYTN,MXZLO
3,AA191175561416,OIEQNT,2019-01-29T18:10:07.000Z,114.956737,22.087467,Y7540547327,35,11630,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wect,CNYTN,MXZLO
4,AA191175561416,OIEQNT,2019-01-29T19:05:02.000Z,115.252152,22.057343,Y7540547327,34,8640,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wecv,CNYTN,MXZLO
5,AA191175561416,OIEQNT,2019-01-29T19:59:24.000Z,115.554182,22.087387,Y7540547327,34,8690,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wefj,CNYTN,MXZLO
6,AA191175561416,OIEQNT,2019-01-29T20:58:24.000Z,115.879635,22.107373,Y7540547327,34,8660,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wefm,CNYTN,MXZLO
7,AA191175561416,OIEQNT,2019-01-29T22:00:49.000Z,116.226522,22.132452,Y7540547327,34,8410,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,weft,CNYTN,MXZLO
8,AA191175561416,OIEQNT,2019-01-29T23:02:04.000Z,116.568513,22.154115,Y7540547327,34,9100,NaN,NaN,NaN,Coastal AIS,CNYTN-MXZLO,wefy,CNYTN,MXZLO
9,AA191175561416,OIEQNT,2019-01-30T00:59:46.000Z,117.227630,22.231482,Y7540547327,35,8360,CN YTN >TW KHH,2019-01-30T04:00:00.000Z,under way using engine,Coastal AIS,CNYTN-MXZLO,wegq,CNYTN,MXZLO


In [51]:
geodf.groupby('loadingOrder')['timestamp'].count()

loadingOrder
AA191175561416     20
AA236935066990     45
AA642567891255     34
AA681773043065      5
AB272331390325      2
AB283635056094      2
AB309086787273      5
AB494971654341      7
AB503189741912      8
AC118422359182    111
AC145390283274      3
AC188113754775     22
AC458999327091    167
AC498331004901      2
AD205574515833     90
AD995240210737      3
AD996489556118     77
AE292412870813      6
AE338443959265     36
AF291281245513     26
AF641280380594     53
AF842018574399     54
AF862663000281     20
AG424688892904     64
AH611886522181     42
AH734641844883     96
AH856056393465      2
AH861147242321     21
AH959858555885    219
AI277413217792    109
AI338314636212      2
AI556878934887     56
AI989403658628     25
AI990380503724     50
AJ600775896968     38
AJ624244686836      6
AJ803226266242      7
AJ860096770625      2
AJ916634114427      7
AJ978747126294     68
AK391029586948     34
AK398507576281     12
AK607102237604      3
AK630924839155     50
AK787247595149     

In [53]:
# geoData = pd.read_csv('E:/traindata/only/temp_geo4.csv')
# t1 = geoData.groupby('loadingOrder')['timestamp'].agg(co='count')
# geoData = geoData.merge(t1,on='loadingOrder',how='left')
# geoData = geoData.drop(geoData.loc[geoData.co==1].index)
# geoData = geoData.reset_index(drop=1)
geodf.shape

(127914, 16)

In [31]:
geoData.to_csv('E:/traindata/only/temp_geo4.csv',index=False)

2118

In [7]:
onlygeo = geoData[['loadingOrder','geo']]
# onlygeo.to_csv('E:/traindata/only/olgeo_tes.csv',index=0)
# onlygeo.to_csv('E:/traindata/only/olgeo_tra.csv',index=0)